In [8]:
import numpy as np
import pandas as pd
import os
import rasterio
from GeoDS import hypercube
from GeoDS.prospectivity import hyperparameterstuning
from GeoDS import utilities
from GeoDS.supervised import mapclass
from GeoDS.prospectivity import reporting 
from GeoDS.prospectivity import featureimportance as fe
from GeoDS import eda
from GeoDS import datawrangle
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from joblib import dump, load
import glob
from dask import dataframe as dd

import optuna
from optuna import pruners
from imblearn.pipeline import Pipeline

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, f1_score, precision_score
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt

#import tensorflow as tf
#import tensorflow_data_validation as tfdv
#from tensorflow_metadata.proto.v0 import schema_pb2

#print('TFDV Version: {}'.format(tfdv.__version__))
#print('Tensorflow Version: {}'.format(tf.__version__))

plt.rcParams["figure.facecolor"] = 'white'
plt.rcParams["axes.facecolor"] = 'white'
plt.rcParams["savefig.facecolor"] = 'white'

In [2]:
crs = 'epsg:26918'
AOI = 'Inputs/AOI/shape/AOI_geol.shp'
xRes = 5
yRes = 5
pixel_size = 5

# Random seed
random_state = 42

In [3]:
trial_name = 'Baseline_Model'

reporting_folder = os.path.join(trial_name, 'reporting/')
output_folder = os.path.join(trial_name, 'outputs/')
predictions_folder = os.path.join(trial_name, 'predictions/')
    
if not os.path.exists(reporting_folder):
    os.makedirs(reporting_folder)
        
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

if not os.path.exists(predictions_folder):
    os.makedirs(predictions_folder)   

In [9]:
df_features = dd.read_csv(os.path.join(output_folder, 'features-*.csv'))
df_features.head()

,x,y,STructural_interpretation_Ludo_buffers_v2_dissolve_for_ML-shp,Critical_DTM_Prelim_Mosaic_modified,All_geology,STR_F_m,New M4 all_20210503,Metamorphism,Critical_Magres_Prelim_Mosaic,Critical_FVD_Prelim_Mosaic,Critical_SVD_Prelim_Mosaic,Pegmatites_All_MERN_Derived from outcrops_20210430,DEM
0,384727.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,384732.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,384737.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,384742.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,384747.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_features = df_features.compute()
df_features.head()

,x,y,STructural_interpretation_Ludo_buffers_v2_dissolve_for_ML-shp,Critical_DTM_Prelim_Mosaic_modified,All_geology,STR_F_m,New M4 all_20210503,Metamorphism,Critical_Magres_Prelim_Mosaic,Critical_FVD_Prelim_Mosaic,Critical_SVD_Prelim_Mosaic,Pegmatites_All_MERN_Derived from outcrops_20210430,DEM
0,384727.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,384732.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,384737.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,384742.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,384747.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_targets = dd.read_csv(os.path.join(output_folder, 'targets-*.csv'))
df_targets.head()

,x,y,CRE_trench_Li100ppm_10mbuffer_positive,CRE_rock_Li100ppm_10mbuffer_positive,CRE_DDH_Li100ppm_25mdepth_10mbuffer_positive,CRE_trench_Li5ppm_10mbuffer_negative,CRE_rock_Li5ppm_10mbuffer_negative
0,384727.5,5760787.5,NaN,NaN,NaN,NaN,NaN
1,384732.5,5760787.5,NaN,NaN,NaN,NaN,NaN
2,384737.5,5760787.5,NaN,NaN,NaN,NaN,NaN
3,384742.5,5760787.5,NaN,NaN,NaN,NaN,NaN
4,384747.5,5760787.5,NaN,NaN,NaN,NaN,NaN


In [12]:
df_targets = df_targets.compute()
df_targets.head()

,x,y,CRE_trench_Li100ppm_10mbuffer_positive,CRE_rock_Li100ppm_10mbuffer_positive,CRE_DDH_Li100ppm_25mdepth_10mbuffer_positive,CRE_trench_Li5ppm_10mbuffer_negative,CRE_rock_Li5ppm_10mbuffer_negative
0,384727.5,5760787.5,NaN,NaN,NaN,NaN,NaN
1,384732.5,5760787.5,NaN,NaN,NaN,NaN,NaN
2,384737.5,5760787.5,NaN,NaN,NaN,NaN,NaN
3,384742.5,5760787.5,NaN,NaN,NaN,NaN,NaN
4,384747.5,5760787.5,NaN,NaN,NaN,NaN,NaN


In [16]:
from functools import reduce
data_frames = [df_features,
               df_targets
              ]
df_Input = reduce(lambda left,right: pd.merge(left,right,on=['x','y']), data_frames)
df_Input.head()

,x,y,STructural_interpretation_Ludo_buffers_v2_dissolve_for_ML-shp,Critical_DTM_Prelim_Mosaic_modified,All_geology,STR_F_m,New M4 all_20210503,Metamorphism,Critical_Magres_Prelim_Mosaic,Critical_FVD_Prelim_Mosaic,Critical_SVD_Prelim_Mosaic,Pegmatites_All_MERN_Derived from outcrops_20210430,DEM,CRE_trench_Li100ppm_10mbuffer_positive,CRE_rock_Li100ppm_10mbuffer_positive,CRE_DDH_Li100ppm_25mdepth_10mbuffer_positive,CRE_trench_Li5ppm_10mbuffer_negative,CRE_rock_Li5ppm_10mbuffer_negative
0,384727.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,384732.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,384737.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,384742.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,384747.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
sd = dd.from_pandas(df_Input, npartitions=100)
sd.to_csv(os.path.join(output_folder, 'cube-*.csv'), index=False)

['/Demo/Baseline_Model/outputs/cube-00.csv',
 '/Demo/Baseline_Model/outputs/cube-01.csv',
 '/Demo/Baseline_Model/outputs/cube-02.csv',
 '/Demo/Baseline_Model/outputs/cube-03.csv',
 '/Demo/Baseline_Model/outputs/cube-04.csv',
 '/Demo/Baseline_Model/outputs/cube-05.csv',
 '/Demo/Baseline_Model/outputs/cube-06.csv',
 '/Demo/Baseline_Model/outputs/cube-07.csv',
 '/Demo/Baseline_Model/outputs/cube-08.csv',
 '/Demo/Baseline_Model/outputs/cube-09.csv',
 '/Demo/Baseline_Model/outputs/cube-10.csv',
 '/Demo/Baseline_Model/outputs/cube-11.csv',
 '/Demo/Baseline_Model/outputs/cube-12.csv',
 '/Demo/Baseline_Model/outputs/cube-13.csv',
 '/Demo/Baseline_Model/outputs/cube-14.csv',
 '/Demo/Baseline_Model/outputs/cube-15.csv',
 '/Demo/Baseline_Model/outputs/cube-16.csv',
 '/Demo/Baseline_Model/outputs/cube-17.csv',
 '/Demo/Baseline_Model/outputs/cube-18.csv',
 '/Demo/Baseline_Model/outputs/cube-19.csv',
 '/Demo/Baseline_Model/outputs/cube-20.csv',
 '/Demo/Baseline_Model/outputs/cube-21.csv',
 '/Demo/Ba

In [18]:
df_Input

,x,y,STructural_interpretation_Ludo_buffers_v2_dissolve_for_ML-shp,Critical_DTM_Prelim_Mosaic_modified,All_geology,STR_F_m,New M4 all_20210503,Metamorphism,Critical_Magres_Prelim_Mosaic,Critical_FVD_Prelim_Mosaic,Critical_SVD_Prelim_Mosaic,Pegmatites_All_MERN_Derived from outcrops_20210430,DEM,CRE_trench_Li100ppm_10mbuffer_positive,CRE_rock_Li100ppm_10mbuffer_positive,CRE_DDH_Li100ppm_25mdepth_10mbuffer_positive,CRE_trench_Li5ppm_10mbuffer_negative,CRE_rock_Li5ppm_10mbuffer_negative
0,384727.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,384732.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,384737.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,384742.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,384747.5,5760787.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358840493,517892.5,5693437.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358840494,517897.5,5693437.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358840495,517902.5,5693437.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358840496,517907.5,5693437.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
np.unique(df_Input['CRE_trench_Li100ppm_10mbuffer_positive'], return_counts=True)

(array([ 0.,  1., nan]), array([       28,        35, 358840435]))

In [20]:
np.unique(df_Input['CRE_rock_Li100ppm_10mbuffer_positive'], return_counts=True)

(array([ 0.,  1., nan]), array([ 58622065,      1257, 300217176]))

In [21]:
np.unique(df_Input['CRE_DDH_Li100ppm_25mdepth_10mbuffer_positive'], return_counts=True)

(array([ 0.,  1., nan]), array([    21247,       153, 358819098]))

In [22]:
np.unique(df_Input['CRE_trench_Li5ppm_10mbuffer_negative'], return_counts=True)

(array([ 0.,  1., nan]), array([        9,        21, 358840468]))

In [23]:
np.unique(df_Input['CRE_rock_Li5ppm_10mbuffer_negative'], return_counts=True)

(array([ 0.,  1., nan]), array([ 58621128,      2194, 300217176]))